<a href="https://colab.research.google.com/github/Ali-7800/ME471/blob/main/HW5/problem_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Isoparametric shape functions for 1D element

Below is a function <code>shape1D</code> which computes the isoparametric shape functions and derivatives for a linear element.

You could easily add functionality to compute shape functions for higher order elements by coding outputs for the cases where <code>order >= 2</code>.

In [4]:
import numpy as np

# Shape functions
def shape1D(r, order=1):
    '''
    Inputs:
    * r (float): isoparametric coordinate on the interval [-1, 1]
    * order (integer): polynomial intoptional argument.
    ----> optional argument that defaults to 1 if omitted
    Outputs:
    * Nhat (1D numpy array): Shape function values at r
    * DNhat (1D numpy array): Shape function derivative values, d(Nhat)/dr, at r
    '''

    # Linear
    if(order==1):
        Nhat = np.array([(1-r)/2, (1+r)/2])
        DNhat = np.array([-0.5, 0.5])
        return Nhat, DNhat
    # Quadratic
    if(order==2):
        Nhat = np.array([(r-1)*r/2, (1+r)*(1-r),(r+1)*r/2])
        DNhat = np.array([r-0.5, -2*r,r+0.5])
        return Nhat, DNhat
    else:
        print('Warning: order>2 shape functions not implemented')
        return

# Numerical integration of an element load vector

Consider a linear rod element with domain $X^e = [0,1,2]$ Suppose it has a constant area $A = 1$ and a body load $b(x) = x^2$. The load vector for the element can be computed via

\begin{align}
\mathbf{P}^e &= \int_{X_1^e}^{X_2^e} \mathbf{N}^{e^T}(x)b(x)A(x)dx\\
             &= \int_{-1}^1 \mathbf{\hat{N}}^T(r)\hat{b}(r)\hat{A}(r)J_e(r)dr
\end{align}

Using Gauss quadrature, we can compute the integral numerically via

$$
\mathbf{P}^e \approx \sum_{\alpha=1}^{N_g} w_{\alpha}\mathbf{\hat{N}}(r_\alpha)\hat{b}(r_{\alpha})\hat{A}(r_{\alpha})J_e(r_{\alpha})dr
$$

The computation will be exact is the integrand is a polynomial of degree $d$ and the inequality $d \le 2N_g - 1$ is satisfied.For this linear element with the given body load and area, we have $d=5$ and so $N_g =3$ is the mininum number of Gauss quadrature points needed for exact integration.

The code below is set up to compute the integral using 1, 2, or 3 quadrature points depending on the value you set for the variable <code>N_g</code>.

In [ ]:
# Problem data (linear element)
Xe = np.array([2, 2.5])
A = 1
b = lambda x: x**4

# Element load with Ng quadrature points
Ng = 3
if(Ng==3):
    rg = np.sqrt(3/5)*np.array([-1, 0, 1])
    wg = np.array([5/9, 8/9, 5/9])
elif(Ng==2):
    rg = np.sqrt(1/3)*np.array([-1, 1])
    wg = np.array([1, 1])
elif(Ng==1):
    rg = [0]
    wg = [2]
# Initialize load vector
Pe = np.zeros(2)
# Compute quadrature via summation formula
for i in range(Ng):
    r = rg[i]
    w = wg[i]
    Nhat, DNhat = shape1D(r, order=1)
    x = Nhat@Xe
    J = DNhat@Xe
    bval = b(x)
    Pe += w*Nhat*bval*A*J
# Print results:
print(f'Pe with Ng = {Ng}: {Pe}')

Pe with Ng = 3: [5.609375 7.521875]


# Part 1: Numerical integration of an element stiffness matrix

linear rod element with domain $X^e = [0,1,2]$ with area $A(x) = 1$ and a variable modulus $E(x) = 20x$. The stiffness matrix can be computed via

\begin{align}
\mathbf{K}^e &= \int_{X_1^e}^{X_2^e} \mathbf{B}^{e^T}(x)E(x)A(x)\mathbf{B}^{e}(x)dx\\
             &= \int_{-1}^1 \mathbf{\hat{B}^e}^T(r)\hat{E}(r)\hat{A}(r)\mathbf{\hat{B}^e}J_e(r)dr
\end{align}

Using Gauss quadrature, we can compute the integral numerically via

$$
\mathbf{K}^e \approx \sum_{\alpha=1}^{N_g} w_{\alpha}\mathbf{\hat{B}}^{e^T}(r_\alpha)\hat{E}(r_{\alpha})\hat{A}(r_{\alpha})\mathbf{\hat{B}}^{e^T}J_e(r_{\alpha})dr
$$

In this proble, $\hat{E}(r)$ has degree 1, $\hat{A}(r)$ has degree 0, $\mathbf{B}^{e}$ has degree 1, and $J_e(r)$ has degree 0. In total the integrand will have degree $d=1+1+0+1+0=3$, which means it will need at least $Ng\ge\frac{d+1}{2}=2$ points for exact integratiion.
The code below is set up to compute the integral using 2 quadrature points.

In [6]:
# Problem data (linear element)
Xe = np.array([0, 1, 2])
A = lambda x: 1
E = lambda x: 20*x

# Element stiffness with Ng quadrature points
Ng = 2
if(Ng==3):
    rg = np.sqrt(3/5)*np.array([-1, 0, 1])
    wg = np.array([5/9, 8/9, 5/9])
elif(Ng==2):
    rg = np.sqrt(1/3)*np.array([-1, 1])
    wg = np.array([1, 1])
elif(Ng==1):
    rg = [0]
    wg = [2]
# Initialize stiffness matrix
Ke = np.zeros((3, 3))
for i in range(Ng):
    r = rg[i]
    w = wg[i]
    Nhat, DNhat = shape1D(r, order=2)
    x = Nhat@Xe
    J = DNhat@Xe
    B = (1/J)*DNhat
    Eval = E(x)
    Aval = A(x)
    # np.outer computes B^T times B when B is a 1D numpy array
    Ke += w*np.outer(B, B)*Eval*Aval*J
print(f'Ke with Ng = {Ng}:\n {Ke}')


Ke with Ng = 2:
 [[ 10.         -13.33333333   3.33333333]
 [-13.33333333  53.33333333 -40.        ]
 [  3.33333333 -40.          36.66666667]]
